<a href="https://colab.research.google.com/github/kosei-s/MachineLearningPractice/blob/master/chainer_tutorial/chainer_tutorial_note7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.datasets import load_iris

x, t = load_iris(return_X_y=True)
x = x.astype('float32')
t = t.astype('int32')

In [0]:
from chainer.datasets import TupleDataset

dataset = TupleDataset(x, t)

In [0]:
dataset[0]

(array([5.1, 3.5, 1.4, 0.2], dtype=float32), 0)

In [0]:
dataset[:2]

[(array([5.1, 3.5, 1.4, 0.2], dtype=float32), 0),
 (array([4.9, 3. , 1.4, 0.2], dtype=float32), 0)]

In [0]:
dataset[:10:2]

[(array([5.1, 3.5, 1.4, 0.2], dtype=float32), 0),
 (array([4.7, 3.2, 1.3, 0.2], dtype=float32), 0),
 (array([5. , 3.6, 1.4, 0.2], dtype=float32), 0),
 (array([4.6, 3.4, 1.4, 0.3], dtype=float32), 0),
 (array([4.4, 2.9, 1.4, 0.2], dtype=float32), 0)]

In [0]:
from chainer.datasets import split_dataset_random

train_val, test = split_dataset_random(dataset, int(len(dataset) * 0.7), seed=0)
train, valid = split_dataset_random(train_val, int(len(train_val) * 0.7), seed=0)

In [0]:
from chainer.iterators import SerialIterator

train_iter = SerialIterator(train, batch_size=4, repeat=True, shuffle=True)

minibatch = train_iter.next()

minibatch

[(array([6.1, 2.6, 5.6, 1.4], dtype=float32), 2),
 (array([6.1, 2.9, 4.7, 1.4], dtype=float32), 1),
 (array([5.7, 4.4, 1.5, 0.4], dtype=float32), 0),
 (array([6.3, 2.8, 5.1, 1.5], dtype=float32), 2)]

In [0]:
import chainer
import chainer.links as L
import chainer.functions as F

class Net(chainer.Chain):
  
  def __init__(self, n_in=4, n_hidden=3, n_out=3):
    super().__init__()
    with self.init_scope():
      self.l1 = L.Linear(n_in, n_hidden)
      self.l2 = L.Linear(n_hidden, n_hidden)
      self.l3 = L.Linear(n_hidden, n_out)
    
  def forward(self, x):
    h = F.relu(self.l1(x))
    h = F.relu(self.l2(x))
    h = self.l3(x)
    
    return h

net = Net()

In [0]:
from chainer import optimizers
from chainer.optimizer_hooks import WeightDecay

optimizer = optimizers.MomentumSGD(lr=0.001, momentum=0.9)
optimizer.setup(net)

for param in net.params():
  if param.name != 'b':
    param.update_rule.add_hook(WeightDecay(0.0001))

In [0]:
gpu_id = 0
n_epoch = 64
n_batchsize = 50

net.to_gpu(gpu_id)

results_train, results_valid = {}, {}
results_train['loss'], results_train['accuracy'] = [], []
results_valid['loss'], results_valid['accuracy'] = [], []

train_iter.reset()

count = 1

for epoch in range(n_epoch):
  
  while True:
    
    train_batch = train_iter.next()
    
    x_train, t_train = chainer.dataset.concat_examples(train_batch, gpu_id)
    
    y_train = net(x_train.array)
    loss_train = F.softbax_cross_entropy(y_train, t_train)
    acc_train = F.accuracy(y_train, t_train)
    
    net.cleargrads()
    loss_train.backward()
    
    optimizer.update()
    
    count += 1
    
    if train_iter.is_new_epoch:
      with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
        x_valid, t_valid = chainer.dataset.concat_examples(valid, gpu_id)
        y_valid = net(x_valid)
        loss_valid = F.softmax_cross_entropy(y_valid, t_valid)
        acc_valid = F.accuracy(y_valid, t_valid)
          
      loss_train.to_cpu()
      loss_valid.to_cpu()
      acc_train.to_cpu()
      acc_valid.to_cpu()
        
      print('epoch: {}, iteration: {}, loss (train): {:.4f}, loss (valid): {:.4f}, acc (train): {:.4f}, acc(valid): {:.4f}'.format(
                  epoch, count, loss_train.array.mean(), loss_valid.array.mean(), acc_train.array.mean(), acc_valid.array.mean()))
        
      results_train['loss'].append(loss_train.array)
      results_train['accuracy'].append(acc_train.array)
      results_valid['loss'].append(loss_valid.array)
      results_valid['accuracy'].append(acc_valid.array)
        
      break

AttributeError: ignored

In [0]:
!python --version

Python 3.6.8
